In [34]:
import os
from yt_dlp import YoutubeDL

In [35]:
import os
from yt_dlp import YoutubeDL

def download_videos_one_by_one(channel_url, save_path="./", playlist_items='1-2', cookies_path="cookies.txt", max_duration=600):
    """
    从指定 YouTube 频道逐个获取符合条件的视频，下载视频及其缩略图。
    
    参数：
    - channel_url (str): 频道或播放列表的 URL。
    - save_path (str): 下载视频和缩略图保存的根路径。
    - playlist_items (str): 要提取的视频范围，例如 '1-2' 表示前两条。
    - cookies_path (str): 浏览器的 cookies 文件路径。
    - max_duration (int): 视频时长的最大值（秒），超出此时间的视频将被跳过。
    """
    try:
        # Step 1: 提取频道信息
        ydl_opts_info = {
            'quiet': False,  # 显示调试信息
            'extract_flat': True,  # 只提取视频信息，不下载视频
            'playlist_items': playlist_items,  # 选择指定范围的视频
        }
        with YoutubeDL(ydl_opts_info) as ydl:
            info_dict = ydl.extract_info(channel_url, download=False)
            videos = info_dict.get('entries', [])
            
            if not videos:
                print("No videos found.")
                return

        # Step 2: 逐个处理符合条件的视频
        for video in videos:
            video_url = video.get('url')
            video_title = video.get('title')
            video_duration = video.get('duration', 0)

            # 检查视频时长
            if video_duration > max_duration:
                print(f"Skipping video (too long): {video_title}")
                continue

            # 创建以视频标题为名称的文件夹
            sanitized_title = "".join(c for c in video_title if c.isalnum() or c in " -_")
            video_folder = os.path.join(save_path, sanitized_title)
            os.makedirs(video_folder, exist_ok=True)

            print(f"Downloading video: {video_title} ({video_url})")

            # 下载视频
            ydl_opts_video = {
                'outtmpl': os.path.join(video_folder, f"%(title)s.%(ext)s"),  # 视频保存路径
                'format': 'bestvideo+bestaudio/best',                       # 最佳视频+最佳音频
                'cookiesfrombrowser': ('chrome',),                         # 从浏览器加载 cookies
            }

            try:
                with YoutubeDL(ydl_opts_video) as ydl:
                    ydl.download([video_url])
                print(f"Download completed for: {video_title}")
            except Exception as e:
                print(f"Failed to download video: {video_title} ({e})")
                continue

            # 获取视频的缩略图 URL (从 thumbnails 列表中)
            thumbnails = video.get('thumbnails', [])
            if thumbnails:
                # 选择第一个有效的缩略图链接
                thumbnail_url = thumbnails[0]['url']
                try:
                    # 下载缩略图文件
                    thumbnail_path = os.path.join(video_folder, "thumbnail.jpg")
                    ydl_opts_thumbnail = {
                        'outtmpl': thumbnail_path,  # 缩略图保存路径
                    }
                    with YoutubeDL(ydl_opts_thumbnail) as ydl:
                        ydl.download([thumbnail_url])
                    print(f"Thumbnail downloaded for: {video_title}")
                except Exception as e:
                    print(f"Failed to download thumbnail for: {video_title} ({e})")
            else:
                print(f"No thumbnail found for: {video_title}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [36]:
channel_url = "https://www.youtube.com/@NBA/videos"
save_path = "./downloads"
playlist_items = '1-1'  # 下载前 5 个视频及其封面
download_videos_one_by_one(channel_url, save_path, playlist_items)


[youtube:tab] Extracting URL: https://www.youtube.com/@NBA/videos
[youtube:tab] @NBA/videos: Downloading webpage
[download] Downloading playlist: NBA - Videos
[youtube:tab] Playlist NBA - Videos: Downloading 1 items
[download] Downloading item 1 of 1
[download] Finished downloading playlist: NBA - Videos
Extracting cookies from chrome
Extracted 3063 cookies from chrome
[youtube] Extracting URL: https://www.youtube.com/watch?v=1520M22DniM
[youtube] 1520M22DniM: Downloading webpage
[youtube] 1520M22DniM: Downloading ios player API JSON
[youtube] 1520M22DniM: Downloading mweb player API JSON
[youtube] 1520M22DniM: Downloading m3u8 information
[info] 1520M22DniM: Downloading 1 format(s): 248+251
[download] ./downloads/NBAs Top 10 Plays of the Night  November 21 2024/NBA’s Top 10 Plays of the Night ｜ November 21, 2024.webm has already been downloaded
Download completed for: NBA’s Top 10 Plays of the Night | November 21, 2024
[generic] Extracting URL: https://i.ytimg.com/vi/1520M22DniM/hqdef

[info] hqdefault: Downloading 1 format(s): 0
[download] Destination: ./downloads/NBAs Top 10 Plays of the Night  November 21 2024/thumbnail.jpg
[download] 100% of    5.79KiB in 00:00:00 at 10.46KiB/s  
Thumbnail downloaded for: NBA’s Top 10 Plays of the Night | November 21, 2024
